# Projeto 3: "Efeitos de vídeo em tempo real"

In [1]:
import numpy as np

## 1: Quais transformações foram implementadas? Explique como cada matriz de transformação afeta os pontos da imagem. 

Implementamos as seguintes transformações em ordem (lembrando que os usuários customizam os parâmetros de cada transformação com suas teclas):

1. **Translação 2 (T2):**
$$
    T2 = \begin{bmatrix}
    1 & 0 & -\frac{\text{image.shape[0]}}{2} \\
    0 & 1 & -\frac{\text{image.shape[1]}}{2} \\
    0 & 0 & 1
    \end{bmatrix}
    $$
A Matriz acima é a inversa da matriz de translação 1, que é a matriz de translação que move a imagem para o centro da tela. A matriz T2 move a imagem de volta para a origem, para que as outras transformações possam ser aplicadas. Se um pixel estiver na posição (x, y) na imagem original, ele estará na posição (x - image.shape[0]/2, y - image.shape[1]/2) na imagem transformada.

2. **Escala (S):**
    (scale` é o fator de escala)
    $$
    S = \begin{bmatrix}
    \text{scale} & 0 & 0 \\
    0 & \text{scale} & 0 \\
    0 & 0 & 1
    \end{bmatrix}
    $$
A matriz acima multiplica as coordenadas x e y de cada pixel por scale`. Se um pixel estiver na posição (x, y) na imagem original, ele estará na posição (scale * x, scale * y) na imagem transformada. Isso da a impressão de que um "zoom" está sendo aplicado na imagem.

3. **Rotação (R):**
    (`radians` é o ângulo de rotação em radianos)
    $$
    R = \begin{bmatrix}
    \cos(\text{radians}) & -\sin(\text{radians}) & 0 \\
    \sin(\text{radians}) & \cos(\text{radians}) & 0 \\
    0 & 0 & 1
    \end{bmatrix}
    $$
A matriz acima rotaciona cada pixel por `radians` radianos em torno da origem. Se um pixel estiver na posição (x, y) na imagem original, ele estará na posição (x * cos(radians) - y * sin(radians), x * sin(radians) + y * cos(radians)) na imagem transformada. Lembrando que precisamos que o centro da imagem esteja na origem pois o ponto que os pixeis se rotacionam por padrão é a origem, (0, 0).

4. **Translação (T):**
    $$
    T = \begin{bmatrix}
    1 & 0 & \frac{\text{image.shape[0]}}{2} \\
    0 & 1 & \frac{\text{image.shape[1]}}{2} \\
    0 & 0 & 1
    \end{bmatrix}
    $$
A matriz acima leva o centro da imagem de volta para a posição original no centro da tela. Se um pixel estiver na posição (x, y) na imagem original, ele estará na posição (x + image.shape[0]/2, y + image.shape[1]/2) na imagem transformada.

## 2: Explique como funciona a remoção de artefatos e relacione o conceito ao código implementado.

Como vimos no terceiro notebook de alglin, ao realizar transformações em imagens, como rotações, pequenos pontos (ou buracos) podem aparecer na imagem resultante. Esses buracos ocorrem quando pontos na imagem transformada $X_d$ não encontram correspondentes exatos na imagem original $X_o$. Isso é evidente na equação $X_d = A X_o$, onde $X_o$ forma uma grade regular, mas $X_d$ pode não seguir o mesmo padrão, resultando em pontos pretos na imagem final.

Para corrigir isso, implementamos as seguintes etapas:

### 1. **Cálculo da Transformação Inversa:**<br>
   O código emprega a inversa da matriz de transformação $A$, conforme a equação $X_o = A^{-1} X_d$. Isso significa que, em vez de mapear diretamente $X_o$ para $X_d$, calculamos onde os pontos de $X_d$ deveriam estar em $X_o$. 


In [ ]:
Xd = np.linalg.inv(Xd) @ X

Isso evita a criação de buracos, pois cada ponto em \(X_d\) é mapeado de volta para um ponto válido em \(X_o\).

### 2. **Filtragem de Índices**:
   Após o cálculo da transformação inversa, é possível que alguns índices caiam fora dos limites da imagem original. Um filtro é aplicado para garantir que apenas índices válidos sejam considerados, evitando acessos a pixels inexistentes e prevenindo artefatos.

In [ ]:
filtro = (Xd[0,:] >= 0) & (Xd[0,:] < image.shape[0]) & (Xd[1,:] >= 0) & (Xd[1,:] < image.shape[1])

### 3. **Conversão e Filtragem**:
Os índices são então convertidos para inteiros e o filtro é aplicado, garantindo que somente índices válidos e dentro dos limites sejam utilizados. Lembramos que RGBs floats não são validos, e nossos elementos representam cores de píxeis em RGB.

In [ ]:
X = X.astype(int)
Xd = Xd.astype(int)
X = X[:,filtro]
Xd = Xd[:,filtro]

### 4. **Transferência de Pixels**:
Finalmente, os valores dos pixels são transferidos da imagem original para a imagem transformada, preenchendo efetivamente os pontos na imagem resultante e minimizando artefatos.


In [ ]:
image_[X[0,:],X[1,:]] = image[Xd[0,:],Xd[1,:]]

Dessa maneira, o processo descrito ajuda a mitigar os problemas associados às transformações de imagem, garantindo que a imagem resultante seja o mais precisa e clara possível, sem a presença de artefatos visuais indesejados.

## 3: Como foi implementado o processo responsável por salvar o vídeo? Explique o código implementado.

### Inicialização
- Primeiro, a função `cv.VideoCapture(0)` é usada para inicializar a captura de vídeo, pegando o feed da primeira câmera detectada (geralmente a câmera embutida em um laptop ou uma webcam externa).
- A função `cap.set(cv.CAP_PROP_FPS, 45)` tenta definir a taxa de quadros da captura para 45 FPS.

### Configuração de Gravação
- A flag `recording` é definida como `False` para indicar que a gravação não está acontecendo inicialmente.
- O formato `H264` indica formato MP4, então definimos que o nosso output de video seria em mp4 na linha  `fourcc = cv.VideoWriter_fourcc(*'H264')`.

### Loop Principal
- Dentro do loop `while True:`, o código lê um frame da câmera com `ret, frame = cap.read()`.
- O frame é redimensionado e transformado.
- Dependendo da tecla pressionada pelo usuário:
  - A tecla `s` inicia a gravação. Quando a gravação não está ativa (`if not recording:`), a gravação é iniciada. O `VideoWriter` é inicializado com o nome do arquivo, o formato `H264`, o framerate e o tamanho do frame. O flag `recording` é definido como `True`.
  - A tecla `e` para a gravação e salva o vídeo. Se a gravação estiver ativa (`if recording:`), a gravação é parada e o vídeo é salvo. O flag `recording` é definido como `False`.
- Se `recording` for `True`, o frame transformado é escrito no arquivo de vídeo usando `out.write((image_transformada * 255).astype(np.uint8))`.